In [110]:
import pandas as pd
import nltk
import numpy as np



In [83]:
df = pd.read_csv('data_preprocessed.csv')

In [59]:
df.columns

Index(['doc_id', 'Name of the document', 'Institution', 'URL', 'Authors',
       'Affiliates', 'Sector', 'Country', 'Date', 'Keywords',
       'Exclusion criteria', 'Status', 'Label', 'MapAIE (ours)', 'Jobin',
       'Fjeld', 'Tidjon', 'Hagendorff', 'Floridi', 'Zeng (LAIP)',
       'Attard-Frost', 'EP', 'Algorithm watch', 'CE', 'Winfield',
       'EthicalML GitHub', 'all sources', 'Checked by', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'text', 'langue', 'text_processed',
       'tfidf'],
      dtype='object')

In [126]:
df[['Sector', 'Institution','Country','Label','Name of the document']].head(10)

,Sector,Institution,Country,Label,Name of the document
3,NaN,intel,NaN,SPI,Intel Recommends Public Policy Principles for ...
5,NaN,academy of medical royal colleges,UK,SPI field-specific,Artificial intelligence in Healthcare
6,NaN,access now,NaN,NaN,Human rights in the age of Artificial Intellig...
7,civil society,access now,UK,NaN,The Toronto Declaration: Protecting the right ...
8,NaN,accessnow,USA,SPI,Europe’s approach to artificial intelligence: ...
9,NaN,accessnow,USA,SPI,Human Rights Impact Assessments for AI: Analys...
10,NaN,accessnow,USA,meta-analysis,Mapping Regulatory Proposals for AI in Europe
11,NaN,ada lovelace institute,UK,"SPI, issue-specific",Regulating AI in Europe: four problems and fou...
12,NaN,adapt center,Ireland,not ai ethics,Ethics Canvas
13,NaN,advisory board on artificial intelligence and ...,Japan,SPI,Report on Artificial Intelligence and Human So...


In [113]:
df["Institution"] = df['Institution'].replace('.',np.nan) 

In [114]:
df['Institution'] = df['Institution'].str.lower()


In [115]:
df['Institution'] = df['Institution'].str.replace('.', '', regex=False).str.replace('?', '', regex=False)

In [116]:
df['Institution'].isna().sum()

np.int64(4)

In [123]:
df['Institution'] = df['Institution'].str.split(',').str[0].str.strip()
df['Institution'] = df['Institution'].str.split(';').str[0].str.strip()

In [125]:
df = df.dropna(subset=['Institution'])